# NGCC-PCC-DAC model

This is the example model implemented in Julia.

In [1]:
# By David PThierry
using JuMP, Clp, DataFrames, Plots, CSV

# Parameters

Load `DataFrame`s containing parameters

In [2]:
#: Fitted parameters
df_gas = DataFrame(CSV.File("../reg/gas_coeffs.csv"))
df_steam = DataFrame(CSV.File("../reg/steam_coeffs.csv"))
#: costs
df_pow_c = DataFrame(CSV.File("../resources/prices_avg.csv")) #: 9th col)
df_ng_c = DataFrame(CSV.File("../resources/natural_gas_price.csv"))

,U.S. Natural Gas Electric Power Price,Price
,String,Float64
1,Feb 2021,16.29
2,Jan 2021,3.33
3,Dec 2020,3.3
4,Nov 2020,3.08
5,Oct 2020,2.57
6,Sep 2020,2.49
7,Aug 2020,2.47
8,Jul 2020,2.1
9,Jun 2020,2.08


Assign parameters for gas turbine.

In [3]:
bPowGasTeLoad = df_gas[1, 2]
aPowGasTeLoad = df_gas[1, 3]

bFuelEload = df_gas[2, 2]
aFuelEload = df_gas[2, 3]

lbcoToTonneco = 0.4535924 / 1000  #: Scale factor
bEmissFactEload = df_gas[3, 2] * lbcoToTonneco
aEmissFactEload = df_gas[3, 3] * lbcoToTonneco

bPowHpEload = df_gas[4, 2] / 1000  #: To scale the kW to MW
aPowHpEload = df_gas[4, 3] / 1000

bPowIpEload = df_gas[5, 2] / 1000
aPowIpEload = df_gas[5, 3] / 1000

bAuxRateGas = df_gas[6, 2] / 1000
aAuxRateGas = df_gas[6, 3] / 1000

0.058520000000000294

Assign parameters for steam turbines.

In [4]:
#: Steam params
bPowLpEload = df_steam[1, 2] / 1000
aPowLpEload = df_steam[1, 3] / 1000

bCcRebDutyEload = df_steam[2, 2]
aCcRebDutyEload = df_steam[2, 3]

bDacSteaEload = df_steam[3, 2]
aDacSteaEload = df_steam[3, 3]

bAuxRateStea = df_steam[4, 2] / 1000
aAuxRateStea = df_steam[4, 3] / 1000


kwhToMmbtu = 3412.1416416 / 1e+06
aSteaUseRateDacAir = 1944 * kwhToMmbtu
aSteaUseRateDacFlue = 1944 * kwhToMmbtu

aPowUseRateDacAir = 2 * 250 / 1000
aPowUseRateDacFlue = 250 / 1000

mmolcoToTonne = (1/1000) * 44.0095 * (0.000001)

aSorbCo2CapFlue = 1 * mmolcoToTonne
aSorbCo2CapAir = 1 * mmolcoToTonne

aCapRatePcc = 0.5
aSteaUseRatePcc = aSteaUseRateDacFlue * .7
aPowUseRatePcc = aPowUseRateDacFlue * 10

2.5

Horizon length, e.g., 1 - `tHorz` hours

In [5]:
tHorz = 24

24

Random data for pricing.

In [6]:
rnum = rand(Float64, tHorz)  # Random numbers

24-element Vector{Float64}:
 0.21213445272857911
 0.41219831050961875
 0.45005307456725396
 0.24847961344046876
 0.34485839712035005
 0.3067420520083859
 0.6355580004610124
 0.036860306284216326
 0.9625773480713216
 0.8363972262375725
 0.534304636240106
 0.05033306439633645
 0.11866975991559459
 0.12347375598107413
 0.32937141824797256
 0.28634900855990986
 0.8306098229567038
 0.00991531902253584
 0.19033020746820029
 0.5268336604249102
 0.22485087606556586
 0.1413017905121805
 0.5386935177820973
 0.035774131728856684

Assign random price signal.

In [7]:
pow_price = rnum .* (df_pow_c[11, 6] * 1000 / 100)  # Power USD/MWh

24-element Vector{Float64}:
  22.613532660866532
  43.940339900325355
  47.97565774886927
  26.487926792753967
  36.761905133029316
  32.698702744093936
  67.75048284914392
   3.92930864989746
 102.61074530440287
  89.15994431692522
  56.9568742231953
   5.365504664649465
  12.650196407002381
  13.162302387582502
  35.11099318523387
  30.52480431248639
  88.54300712718462
   1.0569730078023205
  20.28920011611015
  56.16046820129542
  23.96910338858932
  15.062770868598442
  57.42472899557157
   3.8135224422961223

Assign price of Natural gas.

In [8]:
std_w_ng1000cuft = (50000 / 2636.4)/50
price_ng_lb = df_ng_c[2, 2] * std_w_ng1000cuft  # Price of NG USD/lb(std)

1.263086026399636

Create model.

In [9]:
m = Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [10]:
aCo2PccCapRate = .1

0.1

In [11]:
aPowUseRateComp = 0.001

0.001

In [12]:
# Cost parameters.
cCostInvCombTurb = 1e+02
cCostInvSteaTurb = 1e+02
cCostInvTransInter = 1e+02
cCostInvPcc = 1e+02
cCostInvDac = 1e+03
cCostInvComp = 1e+01

pEmissionPrice = 20000000 # USD/tonne CO2
pCo2TranspPrice = 1e+01
pCo2Credit = 1e+00

1.0

# Variables

The load of the gas turbine will act as the main degree of freedom.

In [13]:
@variable(m, 60 <= yGasTelecLoad[0:tHorz - 1] <= 100)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{VariableRef}:
 yGasTelecLoad[0]
 yGasTelecLoad[1]
 yGasTelecLoad[2]
 yGasTelecLoad[3]
 yGasTelecLoad[4]
 yGasTelecLoad[5]
 yGasTelecLoad[6]
 yGasTelecLoad[7]
 yGasTelecLoad[8]
 yGasTelecLoad[9]
 yGasTelecLoad[10]
 yGasTelecLoad[11]
 yGasTelecLoad[12]
 yGasTelecLoad[13]
 yGasTelecLoad[14]
 yGasTelecLoad[15]
 yGasTelecLoad[16]
 yGasTelecLoad[17]
 yGasTelecLoad[18]
 yGasTelecLoad[19]
 yGasTelecLoad[20]
 yGasTelecLoad[21]
 yGasTelecLoad[22]
 yGasTelecLoad[23]

In [14]:
#: Gas Turbine
@variable(m, 0 <= xPowGasTur[0:tHorz - 1])
@variable(m, 0 <= xPowGross[0:tHorz - 1])
@variable(m, 0 <= xPowOut[ 0:tHorz - 1])
@variable(m, xAuxPowGasT[0:tHorz - 1] >= 0)
@variable(m, 0 <= xFuel[0:tHorz - 1])
@variable(m, 0 <= xCo2Fuel[0:tHorz - 1])

#: Steam Turbine
@variable(m, 0 <= xPowHp[0:tHorz - 1])
@variable(m, 0 <= xPowIp[0:tHorz - 1])
@variable(m, 0 <= xAuxPowSteaT[0:tHorz - 1])

#: Steam generation CC
@variable(m, 0 <= xPowLp[0:tHorz - 1])
@variable(m, 0 <= xCcRebDuty[0:tHorz - 1])
@variable(m, 0 <= xDacSteaDuty[0:tHorz - 1])
@variable(m, 0 <= xPowSteaTur[0:tHorz - 1])


#: PCC
@variable(m, 0 <= xCo2CapPcc[0:tHorz - 1])
@variable(m, 0 <= xSteaUsePcc[0:tHorz - 1])
@variable(m, 0 <= xPowUsePcc[0:tHorz - 1])
@variable(m, 0 <= xCo2PccOut[0:tHorz - 1])
@variable(m, 0 <= vCo2PccVent[0:tHorz - 1])
@variable(m, 0 <= xCo2DacFlueIn[0:tHorz - 1])
@variable(m, 0 <= xPccSteaSlack[0:tHorz - 1])

# Dac-Flue
@variable(m, 0 <= a0Flue[0:tHorz - 1])  
@variable(m, 0 <= a1Flue[0:tHorz]) # State
@variable(m, 0 <= a2Flue[0:tHorz]) # State

@variable(m, 0 <= aR0Flue[0:tHorz - 1])  # Not-a-State
@variable(m, 0 <= aR1Flue[0:tHorz])  # State
@variable(m, 0 <= sFflue[0:tHorz])  # State
@variable(m, 0 <= sSflue[0:tHorz])  # State

@variable(m, 0 <= vAbsFlue[0:tHorz - 1])  # Input
@variable(m, 0 <= vRegFlue[0:tHorz - 1])  # Input

@variable(m, 0 <= xCo2StorDacFlue[0:tHorz - 1])
@variable(m, 0 <= xCo2CapDacFlue[0:tHorz - 1])
@variable(m, 0 <= xSteaUseDacFlue[0:tHorz - 1])
@variable(m, 0 <= xPowUseDacFlue[0:tHorz - 1])
@variable(m, 0 <= xCo2DacVentFlue[0:tHorz - 1])

# Dac-Air
@variable(m, 0 <= a0Air[0:tHorz - 1])  #
@variable(m, 0 <= a1Air[0:tHorz]) # State
@variable(m, 0 <= a2Air[0:tHorz]) # State

@variable(m, 0 <= aR0Air[0:tHorz - 1])  # Not-a-State
@variable(m, 0 <= aR1Air[0:tHorz])  # State
@variable(m, 0 <= sFair[0:tHorz])  # State
@variable(m, 0 <= sSair[0:tHorz])  # State

@variable(m, 0 <= vAbsAir[0:tHorz - 1])  # Input
@variable(m, 0 <= vRegAir[0:tHorz - 1])  # Input

@variable(m, 0 <= xCo2StorDacAir[0:tHorz - 1])
@variable(m, 0 <= xCo2CapDacAir[0:tHorz - 1])
@variable(m, xSteaUseDacAir[0:tHorz - 1] >= 0)
@variable(m, 0 <= xPowUseDacAir[0:tHorz - 1])
@variable(m, 0 <= xDacSteaSlack[0:tHorz - 1])


# CO2 compression
@variable(m, 0 <= xCo2Comp[0:tHorz - 1])
@variable(m, 0 <= xPowUseComp[0:tHorz - 1])
@variable(m, xCo2Vent[0:tHorz - 1])  # This used to be only positive.
@variable(m, xAuxPow[0:tHorz - 1])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{VariableRef}:
 xAuxPow[0]
 xAuxPow[1]
 xAuxPow[2]
 xAuxPow[3]
 xAuxPow[4]
 xAuxPow[5]
 xAuxPow[6]
 xAuxPow[7]
 xAuxPow[8]
 xAuxPow[9]
 xAuxPow[10]
 xAuxPow[11]
 xAuxPow[12]
 xAuxPow[13]
 xAuxPow[14]
 xAuxPow[15]
 xAuxPow[16]
 xAuxPow[17]
 xAuxPow[18]
 xAuxPow[19]
 xAuxPow[20]
 xAuxPow[21]
 xAuxPow[22]
 xAuxPow[23]

# Constraints

## Gas Turbine

The expressions of this section relate the _load of the gas turbine_ (`yGasTelecLoad`). These
include the most relevant performance indicators: 

* Gas turbine power as function of load.

$\mathtt{xPowGasTur}_t = \mathtt{aPowGasTeLoad} \times \mathtt{yGasTelecLoad}_t +\mathtt{bPowGasTeLoad} $

In [15]:
# Gas Turbine
@constraint(m, powGasTur[i = 0:tHorz - 1], 
            xPowGasTur[i] == aPowGasTeLoad * yGasTelecLoad[i] + bPowGasTeLoad
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powGasTur[0] : -4.769999999999982 yGasTelecLoad[0] + xPowGasTur[0] = 1.5006662579253325e-12
 powGasTur[1] : -4.769999999999982 yGasTelecLoad[1] + xPowGasTur[1] = 1.5006662579253325e-12
 powGasTur[2] : -4.769999999999982 yGasTelecLoad[2] + xPowGasTur[2] = 1.5006662579253325e-12
 powGasTur[3] : -4.769999999999982 yGasTelecLoad[3] + xPowGasTur[3] = 1.5006662579253325e-12
 powGasTur[4] : -4.769999999999982 yGasTelecLoad[4] + xPowGasTur[4] = 1.5006662579253325e-12
 powGasTur[5] : -4.769999999999982 yGasTelecLoad[5] + xPowGasTur[5] = 1.5006662579253325e-12
 powGasTur[6] : -4.76999999999998

* Fuel (Gas)

$\mathtt{xFuel}_t = \mathtt{aFuelEload} \times \mathtt{yGasTelecLoad}_t, +\mathtt{bFuelEload}$

In [16]:
@constraint(m, fuelEq[i = 0:tHorz - 1], 
            xFuel[i] == aFuelEload * yGasTelecLoad[i] + bFuelEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 fuelEq[0] : -1509.52983000001 yGasTelecLoad[0] + xFuel[0] = 55528.325199999184
 fuelEq[1] : -1509.52983000001 yGasTelecLoad[1] + xFuel[1] = 55528.325199999184
 fuelEq[2] : -1509.52983000001 yGasTelecLoad[2] + xFuel[2] = 55528.325199999184
 fuelEq[3] : -1509.52983000001 yGasTelecLoad[3] + xFuel[3] = 55528.325199999184
 fuelEq[4] : -1509.52983000001 yGasTelecLoad[4] + xFuel[4] = 55528.325199999184
 fuelEq[5] : -1509.52983000001 yGasTelecLoad[5] + xFuel[5] = 55528.325199999184
 fuelEq[6] : -1509.52983000001 yGasTelecLoad[6] + xFuel[6] = 55528.325199999184
 fuelEq[7] : -1509.52983000001 

* Emissions

$\mathtt{xCo2GasT}_t = \mathtt{aEmissFactEload} \times \mathtt{yGasTelecLoad}_t + \mathtt{bEmissFactEload}$

In [17]:
@constraint(m, co2FuelEq[i = 0:tHorz - 1], 
            xCo2Fuel[i] == aEmissFactEload * yGasTelecLoad[i] + bEmissFactEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2FuelEq[0] : -1.8943379818504946 yGasTelecLoad[0] + xCo2Fuel[0] = 69.88704903203953
 co2FuelEq[1] : -1.8943379818504946 yGasTelecLoad[1] + xCo2Fuel[1] = 69.88704903203953
 co2FuelEq[2] : -1.8943379818504946 yGasTelecLoad[2] + xCo2Fuel[2] = 69.88704903203953
 co2FuelEq[3] : -1.8943379818504946 yGasTelecLoad[3] + xCo2Fuel[3] = 69.88704903203953
 co2FuelEq[4] : -1.8943379818504946 yGasTelecLoad[4] + xCo2Fuel[4] = 69.88704903203953
 co2FuelEq[5] : -1.8943379818504946 yGasTelecLoad[5] + xCo2Fuel[5] = 69.88704903203953
 co2FuelEq[6] : -1.8943379818504946 yGasTelecLoad[6] + xCo2Fuel[6] = 

* Auxiliar power (for gas turbine).

$\mathtt{xAuxPowGasT}_t = \mathtt{aAuxRateGas} \times \mathtt{yGasTelecLoad}_t + \mathtt{bAuxRateGas}$

In [18]:
@constraint(m, auxPowGasT[i = 0:tHorz - 1],
            xAuxPowGasT[i] == aAuxRateGas * yGasTelecLoad[i] + bAuxRateGas
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 auxPowGasT[0] : -0.058520000000000294 yGasTelecLoad[0] + xAuxPowGasT[0] = -2.401066012680532e-14
 auxPowGasT[1] : -0.058520000000000294 yGasTelecLoad[1] + xAuxPowGasT[1] = -2.401066012680532e-14
 auxPowGasT[2] : -0.058520000000000294 yGasTelecLoad[2] + xAuxPowGasT[2] = -2.401066012680532e-14
 auxPowGasT[3] : -0.058520000000000294 yGasTelecLoad[3] + xAuxPowGasT[3] = -2.401066012680532e-14
 auxPowGasT[4] : -0.058520000000000294 yGasTelecLoad[4] + xAuxPowGasT[4] = -2.401066012680532e-14
 auxPowGasT[5] : -0.058520000000000294 yGasTelecLoad[5] + xAuxPowGasT[5] = -2.401066012680532e-14
 au

## Steam Turbines

* Power from HP turbine.

$\mathtt{xPowHp}_t = \mathtt{aPowHpEload} \times \mathtt{yGasTelecLoad}_t$

In [19]:
@constraint(m, powHpEq[i = 0:tHorz - 1], 
            xPowHp[i] == aPowHpEload * yGasTelecLoad[i] + bPowHpEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powHpEq[0] : -0.34072688529999917 yGasTelecLoad[0] + xPowHp[0] = 22.45427125600007
 powHpEq[1] : -0.34072688529999917 yGasTelecLoad[1] + xPowHp[1] = 22.45427125600007
 powHpEq[2] : -0.34072688529999917 yGasTelecLoad[2] + xPowHp[2] = 22.45427125600007
 powHpEq[3] : -0.34072688529999917 yGasTelecLoad[3] + xPowHp[3] = 22.45427125600007
 powHpEq[4] : -0.34072688529999917 yGasTelecLoad[4] + xPowHp[4] = 22.45427125600007
 powHpEq[5] : -0.34072688529999917 yGasTelecLoad[5] + xPowHp[5] = 22.45427125600007
 powHpEq[6] : -0.34072688529999917 yGasTelecLoad[6] + xPowHp[6] = 22.45427125600007
 po

* Power from IP turbine.

$\mathtt{xPowIp}_t = \mathtt{aPowIpEload} \times \mathtt{yGasTelecLoad}_t$


In [20]:
@constraint(m, powIpEq[i = 0:tHorz - 1], 
            xPowIp[i] == aPowIpEload * yGasTelecLoad[i] + bPowIpEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powIpEq[0] : -0.4702062453000054 yGasTelecLoad[0] + xPowIp[0] = 26.159721341999568
 powIpEq[1] : -0.4702062453000054 yGasTelecLoad[1] + xPowIp[1] = 26.159721341999568
 powIpEq[2] : -0.4702062453000054 yGasTelecLoad[2] + xPowIp[2] = 26.159721341999568
 powIpEq[3] : -0.4702062453000054 yGasTelecLoad[3] + xPowIp[3] = 26.159721341999568
 powIpEq[4] : -0.4702062453000054 yGasTelecLoad[4] + xPowIp[4] = 26.159721341999568
 powIpEq[5] : -0.4702062453000054 yGasTelecLoad[5] + xPowIp[5] = 26.159721341999568
 powIpEq[6] : -0.4702062453000054 yGasTelecLoad[6] + xPowIp[6] = 26.159721341999568
 po

* Auxiliary power for Steam turbines

$\mathtt{xAuxPowSteaT}_t = \mathtt{aAuxRateStea} \times\mathtt{yGasTelecLoad}_t$

In [21]:
@constraint(m, auxPowSteaTEq[i = 0:tHorz - 1],
            xAuxPowSteaT[i] == aAuxRateStea * yGasTelecLoad[i] + bAuxRateStea 
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 auxPowSteaTEq[0] : -0.032935694230000125 yGasTelecLoad[0] + xAuxPowSteaT[0] = 2.0083757979999906
 auxPowSteaTEq[1] : -0.032935694230000125 yGasTelecLoad[1] + xAuxPowSteaT[1] = 2.0083757979999906
 auxPowSteaTEq[2] : -0.032935694230000125 yGasTelecLoad[2] + xAuxPowSteaT[2] = 2.0083757979999906
 auxPowSteaTEq[3] : -0.032935694230000125 yGasTelecLoad[3] + xAuxPowSteaT[3] = 2.0083757979999906
 auxPowSteaTEq[4] : -0.032935694230000125 yGasTelecLoad[4] + xAuxPowSteaT[4] = 2.0083757979999906
 auxPowSteaTEq[5] : -0.032935694230000125 yGasTelecLoad[5] + xAuxPowSteaT[5] = 2.0083757979999906
 au

## LP turbine and steam generation

* Power from LP turbine (configuration 1)

$\mathtt{xPowLp}_t = \mathtt{aPowLpEload} \times \mathtt{yGasTelecLoad}_t$

In [22]:
@constraint(m, powLpEq[i = 0:tHorz - 1], 
            xPowLp[i] == aPowLpEload * yGasTelecLoad[i] + bPowLpEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powLpEq[0] : -0.4558890884999999 yGasTelecLoad[0] + xPowLp[0] = 15.91436314200001
 powLpEq[1] : -0.4558890884999999 yGasTelecLoad[1] + xPowLp[1] = 15.91436314200001
 powLpEq[2] : -0.4558890884999999 yGasTelecLoad[2] + xPowLp[2] = 15.91436314200001
 powLpEq[3] : -0.4558890884999999 yGasTelecLoad[3] + xPowLp[3] = 15.91436314200001
 powLpEq[4] : -0.4558890884999999 yGasTelecLoad[4] + xPowLp[4] = 15.91436314200001
 powLpEq[5] : -0.4558890884999999 yGasTelecLoad[5] + xPowLp[5] = 15.91436314200001
 powLpEq[6] : -0.4558890884999999 yGasTelecLoad[6] + xPowLp[6] = 15.91436314200001
 powLpEq[7

* Overall power generated from the turbines.

$\mathtt{xPowSteaTur}_t = \mathtt{xPowHp}_t + \mathtt{xPowIp}_t + \mathtt{xPowLp}_t$

In [23]:
@constraint(m, powerSteaEq[i = 0:tHorz - 1], 
            xPowSteaTur[i] == xPowHp[i] + xPowIp[i] + xPowLp[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powerSteaEq[0] : -xPowHp[0] - xPowIp[0] - xPowLp[0] + xPowSteaTur[0] = 0.0
 powerSteaEq[1] : -xPowHp[1] - xPowIp[1] - xPowLp[1] + xPowSteaTur[1] = 0.0
 powerSteaEq[2] : -xPowHp[2] - xPowIp[2] - xPowLp[2] + xPowSteaTur[2] = 0.0
 powerSteaEq[3] : -xPowHp[3] - xPowIp[3] - xPowLp[3] + xPowSteaTur[3] = 0.0
 powerSteaEq[4] : -xPowHp[4] - xPowIp[4] - xPowLp[4] + xPowSteaTur[4] = 0.0
 powerSteaEq[5] : -xPowHp[5] - xPowIp[5] - xPowLp[5] + xPowSteaTur[5] = 0.0
 powerSteaEq[6] : -xPowHp[6] - xPowIp[6] - xPowLp[6] + xPowSteaTur[6] = 0.0
 powerSteaEq[7] : -xPowHp[7] - xPowIp[7] - xPowLp[7] + xPow

* Available heat for the PCC reboiler.

$\mathtt{xCcRebDuty}_t = \mathtt{aCcRebDutyEload} \times \mathtt{yGasTelecLoad}_t $

In [24]:
@constraint(m, ccRebDutyEq[i = 0:tHorz - 1],
            xCcRebDuty[i] == aCcRebDutyEload * yGasTelecLoad[i] + bCcRebDutyEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 ccRebDutyEq[0] : -3.7200000000000233 yGasTelecLoad[0] + xCcRebDuty[0] = 335.59999999999815
 ccRebDutyEq[1] : -3.7200000000000233 yGasTelecLoad[1] + xCcRebDuty[1] = 335.59999999999815
 ccRebDutyEq[2] : -3.7200000000000233 yGasTelecLoad[2] + xCcRebDuty[2] = 335.59999999999815
 ccRebDutyEq[3] : -3.7200000000000233 yGasTelecLoad[3] + xCcRebDuty[3] = 335.59999999999815
 ccRebDutyEq[4] : -3.7200000000000233 yGasTelecLoad[4] + xCcRebDuty[4] = 335.59999999999815
 ccRebDutyEq[5] : -3.7200000000000233 yGasTelecLoad[5] + xCcRebDuty[5] = 335.59999999999815
 ccRebDutyEq[6] : -3.7200000000000233 y

* Steam generated for DAC.

$\mathtt{xDacSteaDuty}_t = \mathtt{aDacSteaEload} \times \mathtt{yGasTelecLoad}_t$

In [25]:
@constraint(m, dacSteaDutyEq[i = 0:tHorz - 1],
            xDacSteaDuty[i] == aDacSteaEload * yGasTelecLoad[i] + bDacSteaEload
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 dacSteaDutyEq[0] : -1.3386120679999969 yGasTelecLoad[0] + xDacSteaDuty[0] = 55.04318128000024
 dacSteaDutyEq[1] : -1.3386120679999969 yGasTelecLoad[1] + xDacSteaDuty[1] = 55.04318128000024
 dacSteaDutyEq[2] : -1.3386120679999969 yGasTelecLoad[2] + xDacSteaDuty[2] = 55.04318128000024
 dacSteaDutyEq[3] : -1.3386120679999969 yGasTelecLoad[3] + xDacSteaDuty[3] = 55.04318128000024
 dacSteaDutyEq[4] : -1.3386120679999969 yGasTelecLoad[4] + xDacSteaDuty[4] = 55.04318128000024
 dacSteaDutyEq[5] : -1.3386120679999969 yGasTelecLoad[5] + xDacSteaDuty[5] = 55.04318128000024
 dacSteaDutyEq[6] : -

## PCC

* Capacity of the PCC

$\mathtt{xCO2CapPcc}_t = \mathtt{pCo2PccCapRate} \times \mathtt{xCo2GasT}_t$

In [26]:
@constraint(m, co2CapPccEq[i = 0:tHorz - 1], 
            xCo2CapPcc[i] == aCapRatePcc * xCo2Fuel[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2CapPccEq[0] : -0.5 xCo2Fuel[0] + xCo2CapPcc[0] = 0.0
 co2CapPccEq[1] : -0.5 xCo2Fuel[1] + xCo2CapPcc[1] = 0.0
 co2CapPccEq[2] : -0.5 xCo2Fuel[2] + xCo2CapPcc[2] = 0.0
 co2CapPccEq[3] : -0.5 xCo2Fuel[3] + xCo2CapPcc[3] = 0.0
 co2CapPccEq[4] : -0.5 xCo2Fuel[4] + xCo2CapPcc[4] = 0.0
 co2CapPccEq[5] : -0.5 xCo2Fuel[5] + xCo2CapPcc[5] = 0.0
 co2CapPccEq[6] : -0.5 xCo2Fuel[6] + xCo2CapPcc[6] = 0.0
 co2CapPccEq[7] : -0.5 xCo2Fuel[7] + xCo2CapPcc[7] = 0.0
 co2CapPccEq[8] : -0.5 xCo2Fuel[8] + xCo2CapPcc[8] = 0.0
 co2CapPccEq[9] : -0.5 xCo2Fuel[9] + xCo2CapPcc[9] = 0.0
 co2CapPccEq[10] : -0

* Amount of outlet CO2 from PCC

$\mathtt{xCo2PccOut}_t = \mathtt{xCo2Fuel}_{t} - \mathtt{xCo2CapPcc}_{t}$

In [27]:
@constraint(m, co2PccOutEq[i = 0:tHorz - 1], 
            xCo2PccOut[i] == xCo2Fuel[i] - xCo2CapPcc[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2PccOutEq[0] : -xCo2Fuel[0] + xCo2CapPcc[0] + xCo2PccOut[0] = 0.0
 co2PccOutEq[1] : -xCo2Fuel[1] + xCo2CapPcc[1] + xCo2PccOut[1] = 0.0
 co2PccOutEq[2] : -xCo2Fuel[2] + xCo2CapPcc[2] + xCo2PccOut[2] = 0.0
 co2PccOutEq[3] : -xCo2Fuel[3] + xCo2CapPcc[3] + xCo2PccOut[3] = 0.0
 co2PccOutEq[4] : -xCo2Fuel[4] + xCo2CapPcc[4] + xCo2PccOut[4] = 0.0
 co2PccOutEq[5] : -xCo2Fuel[5] + xCo2CapPcc[5] + xCo2PccOut[5] = 0.0
 co2PccOutEq[6] : -xCo2Fuel[6] + xCo2CapPcc[6] + xCo2PccOut[6] = 0.0
 co2PccOutEq[7] : -xCo2Fuel[7] + xCo2CapPcc[7] + xCo2PccOut[7] = 0.0
 co2PccOutEq[8] : -xCo2Fuel[8] + xCo2Ca

* Amount of CO2 that goes into the DAC

$\mathtt{xCo2DacFlueIn}_{t} = \mathtt{xCo2PccOut}_t - \mathtt{yCo2PccVent}_t$

In [28]:
@constraint(m, co2DacFlueInEq[i = 0:tHorz - 1], 
            xCo2DacFlueIn[i] == xCo2PccOut[i] - vCo2PccVent[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2DacFlueInEq[0] : -xCo2PccOut[0] + vCo2PccVent[0] + xCo2DacFlueIn[0] = 0.0
 co2DacFlueInEq[1] : -xCo2PccOut[1] + vCo2PccVent[1] + xCo2DacFlueIn[1] = 0.0
 co2DacFlueInEq[2] : -xCo2PccOut[2] + vCo2PccVent[2] + xCo2DacFlueIn[2] = 0.0
 co2DacFlueInEq[3] : -xCo2PccOut[3] + vCo2PccVent[3] + xCo2DacFlueIn[3] = 0.0
 co2DacFlueInEq[4] : -xCo2PccOut[4] + vCo2PccVent[4] + xCo2DacFlueIn[4] = 0.0
 co2DacFlueInEq[5] : -xCo2PccOut[5] + vCo2PccVent[5] + xCo2DacFlueIn[5] = 0.0
 co2DacFlueInEq[6] : -xCo2PccOut[6] + vCo2PccVent[6] + xCo2DacFlueIn[6] = 0.0
 co2DacFlueInEq[7] : -xCo2PccOut[7] + vCo2Pcc

* Steam used by the PCC reboiler.

$\mathtt{xSteaUsePcc}_t = \mathtt{aSteaUseRatePcc} \times \mathtt{xCo2CapPcc}_t$

In [29]:
@constraint(m, steamUsePccEq[i = 0:tHorz - 1], 
            xSteaUsePcc[i] == aSteaUseRatePcc * xCo2CapPcc[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 steamUsePccEq[0] : -4.64324234588928 xCo2CapPcc[0] + xSteaUsePcc[0] = 0.0
 steamUsePccEq[1] : -4.64324234588928 xCo2CapPcc[1] + xSteaUsePcc[1] = 0.0
 steamUsePccEq[2] : -4.64324234588928 xCo2CapPcc[2] + xSteaUsePcc[2] = 0.0
 steamUsePccEq[3] : -4.64324234588928 xCo2CapPcc[3] + xSteaUsePcc[3] = 0.0
 steamUsePccEq[4] : -4.64324234588928 xCo2CapPcc[4] + xSteaUsePcc[4] = 0.0
 steamUsePccEq[5] : -4.64324234588928 xCo2CapPcc[5] + xSteaUsePcc[5] = 0.0
 steamUsePccEq[6] : -4.64324234588928 xCo2CapPcc[6] + xSteaUsePcc[6] = 0.0
 steamUsePccEq[7] : -4.64324234588928 xCo2CapPcc[7] + xSteaUsePcc[

* Amount of power consumed by the PCC



In [30]:
@constraint(m, powerUsePccEq[i = 0:tHorz - 1], 
            xPowUsePcc[i] == aPowUseRatePcc * xCo2CapPcc[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powerUsePccEq[0] : -2.5 xCo2CapPcc[0] + xPowUsePcc[0] = 0.0
 powerUsePccEq[1] : -2.5 xCo2CapPcc[1] + xPowUsePcc[1] = 0.0
 powerUsePccEq[2] : -2.5 xCo2CapPcc[2] + xPowUsePcc[2] = 0.0
 powerUsePccEq[3] : -2.5 xCo2CapPcc[3] + xPowUsePcc[3] = 0.0
 powerUsePccEq[4] : -2.5 xCo2CapPcc[4] + xPowUsePcc[4] = 0.0
 powerUsePccEq[5] : -2.5 xCo2CapPcc[5] + xPowUsePcc[5] = 0.0
 powerUsePccEq[6] : -2.5 xCo2CapPcc[6] + xPowUsePcc[6] = 0.0
 powerUsePccEq[7] : -2.5 xCo2CapPcc[7] + xPowUsePcc[7] = 0.0
 powerUsePccEq[8] : -2.5 xCo2CapPcc[8] + xPowUsePcc[8] = 0.0
 powerUsePccEq[9] : -2.5 xCo2CapPcc[9] + x

* Remaining Steam available for the PCC

$\mathtt{xPowUsePcc}_t = \mathtt{aPowUseRatePcc} \times \mathtt{xCo2CapPcc}_t$

In [31]:
@constraint(m, pccSteaSlack[i = 0:tHorz - 1], 
            xPccSteaSlack[i] == xCcRebDuty[i] - xSteaUsePcc[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 pccSteaSlack[0] : -xCcRebDuty[0] + xSteaUsePcc[0] + xPccSteaSlack[0] = 0.0
 pccSteaSlack[1] : -xCcRebDuty[1] + xSteaUsePcc[1] + xPccSteaSlack[1] = 0.0
 pccSteaSlack[2] : -xCcRebDuty[2] + xSteaUsePcc[2] + xPccSteaSlack[2] = 0.0
 pccSteaSlack[3] : -xCcRebDuty[3] + xSteaUsePcc[3] + xPccSteaSlack[3] = 0.0
 pccSteaSlack[4] : -xCcRebDuty[4] + xSteaUsePcc[4] + xPccSteaSlack[4] = 0.0
 pccSteaSlack[5] : -xCcRebDuty[5] + xSteaUsePcc[5] + xPccSteaSlack[5] = 0.0
 pccSteaSlack[6] : -xCcRebDuty[6] + xSteaUsePcc[6] + xPccSteaSlack[6] = 0.0
 pccSteaSlack[7] : -xCcRebDuty[7] + xSteaUsePcc[7] + xPccSt

## DAC equations

The following equations represent the state space simple version of the DAC.

* Amount allocated for absorption $\mathtt{a0}_t = {\mathtt{vAbs}_{t}} $.
* Amount allocated for regeneration $\mathtt{aR0}_t = \mathtt{vReg}_{t}$.
* Amount in absorption for one hour $\mathtt{a1}_{t+1} = \mathtt{a0}_{t}$.
* Amount in absorption for two hours $\mathtt{a2}_{t+1}  = \mathtt{a1}_{t}$.
* Amount in regeneration for one hour $\mathtt{aR1}_{t+1} = \mathtt{aR0}_{t}$.
* Amount in fresh inventory $\mathtt{sF}_{t+1} = \mathtt{sF}_{t} - \mathtt{vAbs}_{t} + \mathtt{aR1}_{t}$
* Amount in saturated inventory $\mathtt{sS}_{t+1} = \mathtt{sS}_{t} - \mathtt{vReg}_{t} + \mathtt{a2}_{t}$

In [32]:
# DAC-Flue
@constraint(m, a0FlueEq[i = 0:tHorz - 1], a0Flue[i] == vAbsFlue[i])
@constraint(m, aR0FlueEq[i = 0:tHorz - 1], aR0Flue[i] == vRegFlue[i])
@constraint(m, a1dFlueEq[i = 0:tHorz - 1], a1Flue[i + 1] == a0Flue[i])
@constraint(m, a2dFlueEq[i = 0:tHorz - 1], a2Flue[i + 1] == a1Flue[i])
@constraint(m, aRdFlueEq[i = 0:tHorz - 1], aR1Flue[i + 1] == aR0Flue[i])
@constraint(m, storeFflueeq[i = 0:tHorz - 1], sFflue[i + 1] == sFflue[i] - vAbsFlue[i] + aR1Flue[i])
@constraint(m, storeSflueeq[i = 0:tHorz - 1], sSflue[i + 1] == sSflue[i] - vRegFlue[i] + a2Flue[i])
@constraint(m, capDacFlueEq, sFflue[0] == 1000000)

capDacFlueEq : sFflue[0] = 1.0e6

* Initial Conditions

In [33]:
@constraint(m, icA1FlueEq, a1Flue[0] == 0.)
@constraint(m, icA2FlueEq, a2Flue[0] == 0.)
@constraint(m, icAR1FlueEq, aR1Flue[0] == 0.)
@constraint(m, icSsFlueEq, sSflue[0] == 0.)

icSsFlueEq : sSflue[0] = 0.0

* CO2 storage capacity

In [34]:
@constraint(m, co2StorDacFlueEq[i = 0:tHorz - 1], xCo2StorDacFlue[i] == aSorbCo2CapFlue * sSflue[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2StorDacFlueEq[0] : -4.400950000000001e-8 sSflue[0] + xCo2StorDacFlue[0] = 0.0
 co2StorDacFlueEq[1] : -4.400950000000001e-8 sSflue[1] + xCo2StorDacFlue[1] = 0.0
 co2StorDacFlueEq[2] : -4.400950000000001e-8 sSflue[2] + xCo2StorDacFlue[2] = 0.0
 co2StorDacFlueEq[3] : -4.400950000000001e-8 sSflue[3] + xCo2StorDacFlue[3] = 0.0
 co2StorDacFlueEq[4] : -4.400950000000001e-8 sSflue[4] + xCo2StorDacFlue[4] = 0.0
 co2StorDacFlueEq[5] : -4.400950000000001e-8 sSflue[5] + xCo2StorDacFlue[5] = 0.0
 co2StorDacFlueEq[6] : -4.400950000000001e-8 sSflue[6] + xCo2StorDacFlue[6] = 0.0
 co2StorDacFlueEq

* CO2 capture capacity of the DAC-Flue

In [35]:
@constraint(m, co2CapDacFlueEq[i = 0:tHorz - 1], xCo2CapDacFlue[i] == aSorbCo2CapFlue * aR1Flue[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2CapDacFlueEq[0] : -4.400950000000001e-8 aR1Flue[0] + xCo2CapDacFlue[0] = 0.0
 co2CapDacFlueEq[1] : -4.400950000000001e-8 aR1Flue[1] + xCo2CapDacFlue[1] = 0.0
 co2CapDacFlueEq[2] : -4.400950000000001e-8 aR1Flue[2] + xCo2CapDacFlue[2] = 0.0
 co2CapDacFlueEq[3] : -4.400950000000001e-8 aR1Flue[3] + xCo2CapDacFlue[3] = 0.0
 co2CapDacFlueEq[4] : -4.400950000000001e-8 aR1Flue[4] + xCo2CapDacFlue[4] = 0.0
 co2CapDacFlueEq[5] : -4.400950000000001e-8 aR1Flue[5] + xCo2CapDacFlue[5] = 0.0
 co2CapDacFlueEq[6] : -4.400950000000001e-8 aR1Flue[6] + xCo2CapDacFlue[6] = 0.0
 co2CapDacFlueEq[7] : -4

* Steam used for regeneration

In [36]:
@constraint(m, steamUseDacFlueEq[i = 0:tHorz - 1], xSteaUseDacFlue[i] == aSteaUseRateDacFlue * xCo2CapDacFlue[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 steamUseDacFlueEq[0] : -6.6332033512704 xCo2CapDacFlue[0] + xSteaUseDacFlue[0] = 0.0
 steamUseDacFlueEq[1] : -6.6332033512704 xCo2CapDacFlue[1] + xSteaUseDacFlue[1] = 0.0
 steamUseDacFlueEq[2] : -6.6332033512704 xCo2CapDacFlue[2] + xSteaUseDacFlue[2] = 0.0
 steamUseDacFlueEq[3] : -6.6332033512704 xCo2CapDacFlue[3] + xSteaUseDacFlue[3] = 0.0
 steamUseDacFlueEq[4] : -6.6332033512704 xCo2CapDacFlue[4] + xSteaUseDacFlue[4] = 0.0
 steamUseDacFlueEq[5] : -6.6332033512704 xCo2CapDacFlue[5] + xSteaUseDacFlue[5] = 0.0
 steamUseDacFlueEq[6] : -6.6332033512704 xCo2CapDacFlue[6] + xSteaUseDacFlu

* Power used for the DAC operation

In [37]:
@constraint(m, powUseDacFlueEq[i = 0:tHorz - 1], xPowUseDacFlue[i] == aPowUseRateDacFlue * xCo2CapDacFlue[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powUseDacFlueEq[0] : -0.25 xCo2CapDacFlue[0] + xPowUseDacFlue[0] = 0.0
 powUseDacFlueEq[1] : -0.25 xCo2CapDacFlue[1] + xPowUseDacFlue[1] = 0.0
 powUseDacFlueEq[2] : -0.25 xCo2CapDacFlue[2] + xPowUseDacFlue[2] = 0.0
 powUseDacFlueEq[3] : -0.25 xCo2CapDacFlue[3] + xPowUseDacFlue[3] = 0.0
 powUseDacFlueEq[4] : -0.25 xCo2CapDacFlue[4] + xPowUseDacFlue[4] = 0.0
 powUseDacFlueEq[5] : -0.25 xCo2CapDacFlue[5] + xPowUseDacFlue[5] = 0.0
 powUseDacFlueEq[6] : -0.25 xCo2CapDacFlue[6] + xPowUseDacFlue[6] = 0.0
 powUseDacFlueEq[7] : -0.25 xCo2CapDacFlue[7] + xPowUseDacFlue[7] = 0.0
 powUseDacFlueE

* Amount of CO2 vented by DAC

In [38]:
@constraint(m, co2DacFlueVentEq[i = 0:tHorz - 1], xCo2DacVentFlue[i] == xCo2DacFlueIn[i] - xCo2CapDacFlue[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2DacFlueVentEq[0] : -xCo2DacFlueIn[0] + xCo2CapDacFlue[0] + xCo2DacVentFlue[0] = 0.0
 co2DacFlueVentEq[1] : -xCo2DacFlueIn[1] + xCo2CapDacFlue[1] + xCo2DacVentFlue[1] = 0.0
 co2DacFlueVentEq[2] : -xCo2DacFlueIn[2] + xCo2CapDacFlue[2] + xCo2DacVentFlue[2] = 0.0
 co2DacFlueVentEq[3] : -xCo2DacFlueIn[3] + xCo2CapDacFlue[3] + xCo2DacVentFlue[3] = 0.0
 co2DacFlueVentEq[4] : -xCo2DacFlueIn[4] + xCo2CapDacFlue[4] + xCo2DacVentFlue[4] = 0.0
 co2DacFlueVentEq[5] : -xCo2DacFlueIn[5] + xCo2CapDacFlue[5] + xCo2DacVentFlue[5] = 0.0
 co2DacFlueVentEq[6] : -xCo2DacFlueIn[6] + xCo2CapDacFlue[6] + 

### DAC-Air (Similar to DAC-Flue)

In [39]:
@constraint(m, a0AirEq[i = 0:tHorz - 1], a0Air[i] == vAbsAir[i])
@constraint(m, aR0AirEq[i = 0:tHorz - 1], aR0Air[i] == vRegAir[i])
@constraint(m, a1dAirEq[i = 0:tHorz - 1], a1Air[i + 1] == a0Air[i])
@constraint(m, a2dAirEq[i = 0:tHorz - 1], a2Air[i + 1] == a1Air[i])
@constraint(m, aRdAirEq[i = 0:tHorz - 1], aR1Air[i + 1] == aR0Air[i])
@constraint(m, storeFairEq[i = 0:tHorz - 1], sFair[i + 1] == sFair[i] - vAbsAir[i] + aR1Air[i])
@constraint(m, storeSaireq[i = 0:tHorz - 1], sSair[i + 1] == sSair[i] - vRegAir[i] + a2Air[i])
@constraint(m, capDacAirEq, sFair[0] == 1000000)
# 
@constraint(m, icA1AirEq, a1Air[0] == 0.)
@constraint(m, icA2AirEq, a2Air[0] == 0.)
@constraint(m, icAR1AirEq, aR1Air[0] == 0.)
#
@constraint(m, icSsAirEq, sSair[0] == 0.)
# 
@constraint(m, co2StorDacAirEq[i = 0:tHorz - 1], xCo2StorDacAir[i] == aSorbCo2CapAir * sSair[i])
@constraint(m, co2CapDacAirEq[i = 0:tHorz - 1], xCo2CapDacAir[i] == aSorbCo2CapAir * aR1Air[i])
# 
@constraint(m, steamUseDacAirEq[i = 0:tHorz - 1], xSteaUseDacAir[i] == aSteaUseRateDacAir * xCo2CapDacAir[i])
# 
@constraint(m, powUseDacAirEq[i = 0:tHorz - 1], xPowUseDacAir[i] == aPowUseRateDacAir * xCo2CapDacAir[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powUseDacAirEq[0] : -0.5 xCo2CapDacAir[0] + xPowUseDacAir[0] = 0.0
 powUseDacAirEq[1] : -0.5 xCo2CapDacAir[1] + xPowUseDacAir[1] = 0.0
 powUseDacAirEq[2] : -0.5 xCo2CapDacAir[2] + xPowUseDacAir[2] = 0.0
 powUseDacAirEq[3] : -0.5 xCo2CapDacAir[3] + xPowUseDacAir[3] = 0.0
 powUseDacAirEq[4] : -0.5 xCo2CapDacAir[4] + xPowUseDacAir[4] = 0.0
 powUseDacAirEq[5] : -0.5 xCo2CapDacAir[5] + xPowUseDacAir[5] = 0.0
 powUseDacAirEq[6] : -0.5 xCo2CapDacAir[6] + xPowUseDacAir[6] = 0.0
 powUseDacAirEq[7] : -0.5 xCo2CapDacAir[7] + xPowUseDacAir[7] = 0.0
 powUseDacAirEq[8] : -0.5 xCo2CapDacAir[8] + xP

* Amount of Steam available for CO2 Capture

In [40]:
@constraint(m, dacSteaSlackEq[i = 0:tHorz - 1], 
            xDacSteaSlack[i] == xDacSteaDuty[i] - xSteaUseDacFlue[i] - xSteaUseDacAir[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 dacSteaSlackEq[0] : -xDacSteaDuty[0] + xSteaUseDacFlue[0] + xSteaUseDacAir[0] + xDacSteaSlack[0] = 0.0
 dacSteaSlackEq[1] : -xDacSteaDuty[1] + xSteaUseDacFlue[1] + xSteaUseDacAir[1] + xDacSteaSlack[1] = 0.0
 dacSteaSlackEq[2] : -xDacSteaDuty[2] + xSteaUseDacFlue[2] + xSteaUseDacAir[2] + xDacSteaSlack[2] = 0.0
 dacSteaSlackEq[3] : -xDacSteaDuty[3] + xSteaUseDacFlue[3] + xSteaUseDacAir[3] + xDacSteaSlack[3] = 0.0
 dacSteaSlackEq[4] : -xDacSteaDuty[4] + xSteaUseDacFlue[4] + xSteaUseDacAir[4] + xDacSteaSlack[4] = 0.0
 dacSteaSlackEq[5] : -xDacSteaDuty[5] + xSteaUseDacFlue[5] + xSteaUseDa

## Compression CO2

In [41]:
# Co2 Compression
# 27
@constraint(m, co2CompEq[i = 0:tHorz - 1], xCo2Comp[i] == xCo2CapPcc[i])


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2CompEq[0] : -xCo2CapPcc[0] + xCo2Comp[0] = 0.0
 co2CompEq[1] : -xCo2CapPcc[1] + xCo2Comp[1] = 0.0
 co2CompEq[2] : -xCo2CapPcc[2] + xCo2Comp[2] = 0.0
 co2CompEq[3] : -xCo2CapPcc[3] + xCo2Comp[3] = 0.0
 co2CompEq[4] : -xCo2CapPcc[4] + xCo2Comp[4] = 0.0
 co2CompEq[5] : -xCo2CapPcc[5] + xCo2Comp[5] = 0.0
 co2CompEq[6] : -xCo2CapPcc[6] + xCo2Comp[6] = 0.0
 co2CompEq[7] : -xCo2CapPcc[7] + xCo2Comp[7] = 0.0
 co2CompEq[8] : -xCo2CapPcc[8] + xCo2Comp[8] = 0.0
 co2CompEq[9] : -xCo2CapPcc[9] + xCo2Comp[9] = 0.0
 co2CompEq[10] : -xCo2CapPcc[10] + xCo2Comp[10] = 0.0
 co2CompEq[11] : -xCo2CapPc

In [42]:
@constraint(m, powUseCompEq[i = 0:tHorz - 1], xPowUseComp[i] == aPowUseRateComp * xCo2Comp[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powUseCompEq[0] : -0.001 xCo2Comp[0] + xPowUseComp[0] = 0.0
 powUseCompEq[1] : -0.001 xCo2Comp[1] + xPowUseComp[1] = 0.0
 powUseCompEq[2] : -0.001 xCo2Comp[2] + xPowUseComp[2] = 0.0
 powUseCompEq[3] : -0.001 xCo2Comp[3] + xPowUseComp[3] = 0.0
 powUseCompEq[4] : -0.001 xCo2Comp[4] + xPowUseComp[4] = 0.0
 powUseCompEq[5] : -0.001 xCo2Comp[5] + xPowUseComp[5] = 0.0
 powUseCompEq[6] : -0.001 xCo2Comp[6] + xPowUseComp[6] = 0.0
 powUseCompEq[7] : -0.001 xCo2Comp[7] + xPowUseComp[7] = 0.0
 powUseCompEq[8] : -0.001 xCo2Comp[8] + xPowUseComp[8] = 0.0
 powUseCompEq[9] : -0.001 xCo2Comp[9] + xP

In [43]:
@constraint(m, co2VentEq[i = 0:tHorz - 1], xCo2Vent[i] == vCo2PccVent[i] + xCo2DacVentFlue[i] - 
            xCo2CapDacAir[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 co2VentEq[0] : -vCo2PccVent[0] - xCo2DacVentFlue[0] + xCo2CapDacAir[0] + xCo2Vent[0] = 0.0
 co2VentEq[1] : -vCo2PccVent[1] - xCo2DacVentFlue[1] + xCo2CapDacAir[1] + xCo2Vent[1] = 0.0
 co2VentEq[2] : -vCo2PccVent[2] - xCo2DacVentFlue[2] + xCo2CapDacAir[2] + xCo2Vent[2] = 0.0
 co2VentEq[3] : -vCo2PccVent[3] - xCo2DacVentFlue[3] + xCo2CapDacAir[3] + xCo2Vent[3] = 0.0
 co2VentEq[4] : -vCo2PccVent[4] - xCo2DacVentFlue[4] + xCo2CapDacAir[4] + xCo2Vent[4] = 0.0
 co2VentEq[5] : -vCo2PccVent[5] - xCo2DacVentFlue[5] + xCo2CapDacAir[5] + xCo2Vent[5] = 0.0
 co2VentEq[6] : -vCo2PccVent[6] - xCo2D

* Gross power from the plant

In [44]:
@constraint(m, powGrossEq[i = 0:tHorz - 1], 
            xPowGross[i] == xPowGasTur[i] + xPowSteaTur[i]
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powGrossEq[0] : -xPowGasTur[0] + xPowGross[0] - xPowSteaTur[0] = 0.0
 powGrossEq[1] : -xPowGasTur[1] + xPowGross[1] - xPowSteaTur[1] = 0.0
 powGrossEq[2] : -xPowGasTur[2] + xPowGross[2] - xPowSteaTur[2] = 0.0
 powGrossEq[3] : -xPowGasTur[3] + xPowGross[3] - xPowSteaTur[3] = 0.0
 powGrossEq[4] : -xPowGasTur[4] + xPowGross[4] - xPowSteaTur[4] = 0.0
 powGrossEq[5] : -xPowGasTur[5] + xPowGross[5] - xPowSteaTur[5] = 0.0
 powGrossEq[6] : -xPowGasTur[6] + xPowGross[6] - xPowSteaTur[6] = 0.0
 powGrossEq[7] : -xPowGasTur[7] + xPowGross[7] - xPowSteaTur[7] = 0.0
 powGrossEq[8] : -xPowGasTur[8]

* Total auxiliary power

In [45]:
@constraint(m, auxPowEq[i = 0:tHorz - 1],
            xAuxPow[i] == xAuxPowGasT[i] + xAuxPowSteaT[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 auxPowEq[0] : -xAuxPowGasT[0] - xAuxPowSteaT[0] + xAuxPow[0] = 0.0
 auxPowEq[1] : -xAuxPowGasT[1] - xAuxPowSteaT[1] + xAuxPow[1] = 0.0
 auxPowEq[2] : -xAuxPowGasT[2] - xAuxPowSteaT[2] + xAuxPow[2] = 0.0
 auxPowEq[3] : -xAuxPowGasT[3] - xAuxPowSteaT[3] + xAuxPow[3] = 0.0
 auxPowEq[4] : -xAuxPowGasT[4] - xAuxPowSteaT[4] + xAuxPow[4] = 0.0
 auxPowEq[5] : -xAuxPowGasT[5] - xAuxPowSteaT[5] + xAuxPow[5] = 0.0
 auxPowEq[6] : -xAuxPowGasT[6] - xAuxPowSteaT[6] + xAuxPow[6] = 0.0
 auxPowEq[7] : -xAuxPowGasT[7] - xAuxPowSteaT[7] + xAuxPow[7] = 0.0
 auxPowEq[8] : -xAuxPowGasT[8] - xAuxPowSteaT[8

* Total Power out

In [46]:
@constraint(m, powOutEq[i = 0:tHorz - 1], 
            xPowOut[i] == xPowGross[i] - xPowUsePcc[i] - xPowUseDacFlue[i] 
            - xPowUseDacAir[i] - xPowUseComp[i] - xAuxPow[i]
           )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 0:23
And data, a 24-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 powOutEq[0] : -xPowGross[0] + xPowOut[0] + xPowUsePcc[0] + xPowUseDacFlue[0] + xPowUseDacAir[0] + xPowUseComp[0] + xAuxPow[0] = 0.0
 powOutEq[1] : -xPowGross[1] + xPowOut[1] + xPowUsePcc[1] + xPowUseDacFlue[1] + xPowUseDacAir[1] + xPowUseComp[1] + xAuxPow[1] = 0.0
 powOutEq[2] : -xPowGross[2] + xPowOut[2] + xPowUsePcc[2] + xPowUseDacFlue[2] + xPowUseDacAir[2] + xPowUseComp[2] + xAuxPow[2] = 0.0
 powOutEq[3] : -xPowGross[3] + xPowOut[3] + xPowUsePcc[3] + xPowUseDacFlue[3] + xPowUseDacAir[3] + xPowUseComp[3] + xAuxPow[3] = 0.0
 powOutEq[4] : -xPowGross[4] + xPowOut[4] + xPowUsePcc[4] +

# Objective function

In [47]:
@expression(m, eObjfExpr, sum(price_ng_lb * xFuel[i] + 
                              pEmissionPrice * xCo2Vent[i] + 
                              pCo2TranspPrice * xCo2Comp[i] - 
                              pow_price[i+1] * xPowOut[i] for i in 0:tHorz - 1))
                              #pPowBasePrice * xPowOut[i] for i in 0:tHorz - 1))
#@expression(m, eObjfExpr, sum(-xPowOut[i] for i in 0:tHorz - 1))

1.263086026399636 xFuel[0] + 2.0e7 xCo2Vent[0] + 10 xCo2Comp[0] - 22.613532660866532 xPowOut[0] + 1.263086026399636 xFuel[1] + 2.0e7 xCo2Vent[1] + 10 xCo2Comp[1] - 43.940339900325355 xPowOut[1] + 1.263086026399636 xFuel[2] + 2.0e7 xCo2Vent[2] + 10 xCo2Comp[2] - 47.97565774886927 xPowOut[2] + 1.263086026399636 xFuel[3] + 2.0e7 xCo2Vent[3] + 10 xCo2Comp[3] - 26.487926792753967 xPowOut[3] + 1.263086026399636 xFuel[4] + 2.0e7 xCo2Vent[4] + 10 xCo2Comp[4] - 36.761905133029316 xPowOut[4] + 1.263086026399636 xFuel[5] + 2.0e7 xCo2Vent[5] + 10 xCo2Comp[5] - 32.698702744093936 xPowOut[5] + 1.263086026399636 xFuel[6] + 2.0e7 xCo2Vent[6] + 10 xCo2Comp[6] - 67.75048284914392 xPowOut[6] + 1.263086026399636 xFuel[7] + 2.0e7 xCo2Vent[7] + 10 xCo2Comp[7] - 3.92930864989746 xPowOut[7] + 1.263086026399636 xFuel[8] + 2.0e7 xCo2Vent[8] + 10 xCo2Comp[8] - 102.61074530440287 xPowOut[8] + 1.263086026399636 xFuel[9] + 2.0e7 xCo2Vent[9] + 10 xCo2Comp[9] - 89.15994431692522 xPowOut[9] + 1.263086026399636 xFuel[1

In [48]:
@objective(m, Min, eObjfExpr)

1.263086026399636 xFuel[0] + 2.0e7 xCo2Vent[0] + 10 xCo2Comp[0] - 22.613532660866532 xPowOut[0] + 1.263086026399636 xFuel[1] + 2.0e7 xCo2Vent[1] + 10 xCo2Comp[1] - 43.940339900325355 xPowOut[1] + 1.263086026399636 xFuel[2] + 2.0e7 xCo2Vent[2] + 10 xCo2Comp[2] - 47.97565774886927 xPowOut[2] + 1.263086026399636 xFuel[3] + 2.0e7 xCo2Vent[3] + 10 xCo2Comp[3] - 26.487926792753967 xPowOut[3] + 1.263086026399636 xFuel[4] + 2.0e7 xCo2Vent[4] + 10 xCo2Comp[4] - 36.761905133029316 xPowOut[4] + 1.263086026399636 xFuel[5] + 2.0e7 xCo2Vent[5] + 10 xCo2Comp[5] - 32.698702744093936 xPowOut[5] + 1.263086026399636 xFuel[6] + 2.0e7 xCo2Vent[6] + 10 xCo2Comp[6] - 67.75048284914392 xPowOut[6] + 1.263086026399636 xFuel[7] + 2.0e7 xCo2Vent[7] + 10 xCo2Comp[7] - 3.92930864989746 xPowOut[7] + 1.263086026399636 xFuel[8] + 2.0e7 xCo2Vent[8] + 10 xCo2Comp[8] - 102.61074530440287 xPowOut[8] + 1.263086026399636 xFuel[9] + 2.0e7 xCo2Vent[9] + 10 xCo2Comp[9] - 89.15994431692522 xPowOut[9] + 1.263086026399636 xFuel[1

# Solver declaration and solution

In [49]:
set_optimizer(m, Clp.Optimizer)
set_optimizer_attribute(m, "LogLevel", 3)
set_optimizer_attribute(m, "PresolveType", 1)

1

## Solve

In [50]:
optimize!(m)
println(termination_status(m))

OPTIMAL
Clp1001I Initial range of elements is 4.40095e-08 to 1509.5298
Clp1002I Range of elements is 0.00020978441 to 4766.7984
Clp1002I Range of elements is 0.01217949 to 82.105243
Clp1003I Final range of elements is 0.032194279 to 31.061419
Clp0022I Absolute values of scaled rhs range from 2.3062292e-13 to 1000000, minimum gap 1e+100
Clp0020I Absolute values of scaled objective range from 15.52486 to 81531107
Clp0021I Absolute values of scaled bounds range from 19.620223 to 32.700372, minimum gap 13.080149
Clp0006I 0  Obj 0 Primal inf 2058204.1 (242) Dual inf 19580597 (48) w.o. free dual inf (24)
Clp0060I Primal error 0.5, dual error 0
Clp0006I 97  Obj -3.4239961e+16 Primal inf 1.0762466e+16 (341)
Clp0060I Primal error 0.5, dual error 9.6279381e-19
Clp0006I 194  Obj -3.4239961e+16 Primal inf 1.0226041e+16 (347)
Clp0006I 291  Obj -1.3131386e+14 Primal inf 3.9418985e+12 (280)
Clp0006I 388  Obj -1.3131386e+14 Primal inf 1.1731554e+13 (241)
Clp0006I 485  Obj -6758722.9 Primal inf 183631.

## Report

In [51]:

# Co2 Data Frame
df_co = DataFrame(Symbol("Co2Fuel") => Float64[], # Pairs.
                  Symbol("Co2CapPcc") => Float64[],
                  Symbol("Co2PccOut") => Float64[],
                  Symbol("vCo2PccVent") => Float64[],
                  Symbol("Co2DacFlueIn") => Float64[],
                  Symbol("Co2StoreDacFlue") => Float64[],
                  Symbol("Co2CapDacFlue") => Float64[],
                  Symbol("Co2CapDacAir") => Float64[],
                  Symbol("Co2DacVentFlue") => Float64[],
                  Symbol("Co2Vent") => Float64[],
                 )
for i in 0:tHorz - 1
    push!(df_co, (
                  value(xCo2Fuel[i]),
               value(xCo2CapPcc[i]),
               value(xCo2PccOut[i]), 
               value(vCo2PccVent[i]), 
               value(xCo2DacFlueIn[i]), 
               value(xCo2StorDacFlue[i]), 
               value(xCo2CapDacFlue[i]), 
               value(xCo2CapDacAir[i]), 
               value(xCo2DacVentFlue[i]), 
               value(xCo2Vent[i])))
end

# Power Data Frame.
df_pow = DataFrame(
                  Symbol("PowGasTur") => Float64[], # Pairs.
                  Symbol("PowSteaTurb") => Float64[],
                  Symbol("PowHp") => Float64[],
                  Symbol("PowIp") => Float64[],
                  Symbol("PowLp") => Float64[],
                  Symbol("PowUsePcc") => Float64[],
                  Symbol("PowUseDacFlue") => Float64[],
                  Symbol("PowUseDacAir") => Float64[],
                  Symbol("PowUseComp") => Float64[],
                  Symbol("AuxPowGasT") => Float64[],
                  Symbol("AuxPowSteaT") => Float64[],
                  Symbol("PowGross") => Float64[],
                  Symbol("PowOut") => Float64[],
                  Symbol("yGasTelecLoad") => Float64[],
                  Symbol("Demand") => Float64[]
                 )
for i in 0:tHorz-1
    push!(df_pow, (
                   value(xPowGasTur[i]),
                   value(xPowSteaTur[i]),
                   value(xPowHp[i]), 
                   value(xPowIp[i]), 
                   value(xPowLp[i]), 
                   value(xPowUsePcc[i]), 
                   value(xPowUseDacFlue[i]), 
                   value(xPowUseDacAir[i]), 
                   value(xPowUseComp[i]),
                   value(xAuxPowGasT[i]),
                   value(xAuxPowSteaT[i]),
                   value(xPowGross[i]),
                   value(xPowOut[i]),
                   value(yGasTelecLoad[i]),
                   rnum[i + 1]
                  ))
end

df_steam = DataFrame(
                     Symbol("CcRebDuty") => Float64[],
                     Symbol("SteaUsePcc") => Float64[],
                     Symbol("PccSteaSlack") => Float64[],
                     Symbol("DacSteaDuty") => Float64[],
                     Symbol("SteaUseDacFlue") => Float64[],
                     Symbol("SteaUseDacAir") => Float64[],
                     Symbol("DacSteaSlack") => Float64[],
                     Symbol("Fuel") => Float64[]
                    )
for i in 0:tHorz-1
    push!(df_steam, (
                     value(xCcRebDuty[i]),
                     value(xSteaUsePcc[i]),
                     value(xPccSteaSlack[i]),
                     value(xDacSteaDuty[i]),
                     value(xSteaUseDacFlue[i]),
                     value(xSteaUseDacAir[i]),
                     value(xDacSteaSlack[i]),
                     value(xFuel[i])
                    ),
         )
end

# Generate Dataframe of the DAC-Flue variables.
df_dac_flue = DataFrame(:sFflue => Float64[],
                     :sSflue => Float64[],
                     :vAbsFlue => Float64[],
                     :vRegFlue => Float64[],
                     :a0Flue => Float64[],
                     :a1Flue => Float64[],
                     :a2Flue => Float64[],
                     :aR0Flue => Float64[],
                     :aR1Flue => Float64[])
for i in 0:tHorz - 1
    push!(df_dac_flue,(value(sFflue[i]), value(sSflue[i]),
                  value(vAbsFlue[i]), value(vRegFlue[i]),
                  value(a0Flue[i]), value(a1Flue[i]), value(a2Flue[i]),
                  value(aR0Flue[i]), value(aR1Flue[i])))
end

# Generate Dataframe of the DAC-Air variables.
df_dac_air = DataFrame(:sFair => Float64[],
                     :sSair => Float64[],
                     :vAbsAir => Float64[],
                     :vRegAir => Float64[],
                     :a0Air => Float64[],
                     :a1Air => Float64[],
                     :a2Air => Float64[],
                     :aR0Air => Float64[],
                     :aR1Air => Float64[])
for i in 0:tHorz - 1
    push!(df_dac_air,(value(sFair[i]), value(sSair[i]),
                  value(vAbsAir[i]), value(vRegAir[i]),
                  value(a0Air[i]), value(a1Air[i]), value(a2Air[i]),
                  value(aR0Air[i]), value(aR1Air[i])))
end


df_pow_price = DataFrame(
                         price = Float64[]
                        )
for i in 0:tHorz - 1
    push!(df_pow_price, pow_price[i + 1])
end
          

CSV.write("df_co.csv", df_co)
CSV.write("df_pow.csv", df_pow)
CSV.write("df_steam.csv", df_steam)
CSV.write("df_dac_flue.csv", df_dac_flue)
CSV.write("df_dac_air.csv", df_dac_air)
CSV.write("df_pow_price.csv", df_pow_price)

"df_pow_price.csv"